# Test Dataset Creation

For part of this project, we will develop our own custom testing dataset for twitter sentiment analysis. This dataset will be roughly 500 entries long and will be hand labelled with the following classes: Positive, Neutral, Negative. This notebook will walk the reader through the steps taken to create the test dataset.

## Objective

The objective is to create a high-quality and original dataset for sentiment analysis sourced from the statuses table of the Twitter PostgreSQL Database. We chose to use the PostgreSQl Database instead of the MongoDB Database because the RDBMS table-based system of PostgreSQL is formatted much better to use with AI/ML libraries than the NoSQL JSON format of MongoDB. This is because we can directly query the data from a PostgreSQL table into a Pandas Dataframe, which is the format all of the python AI libraries take. As mentioned before, the dataset will consist of around 500 unique tweets taken from the twitter.statuses table. Each row entry will consist of the full tweet text and a multiclass label (Positive, Neutral, Negative). The dataset size being at 500 is only because this is the minimum length I feel would still produce meaningful results. Larger datasets are always beigger, but since we are hand-labelling the data, we are limited on dataset size. 

## Approach

1. Connect to Twitter PostgreSql database
2. Load 500 entries into a Pandas Dataframe.
3. Reformat data.
4. Export data to a CSV with placeholder labels.
5. Go through dataset by hand and label every entry.
6. Display resulting dataset and show some distributions, etc. 

If you are curious about the actualy sentiment analysis, see the file "SentimentAnalysis.ipynb". I will start by importing the necessary libraries and connecting to the Twitter comp_dbms PostgreSQL 

In [116]:
# Import packages
import pandas as pd
import psycopg2

### Step 1.
Now I will connect to the database and display some basic information about the table twitter.statuses. This is where we will be getting the data to construct the dataset.

In [117]:
# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

## Step 2. 

Now I will load 500 entries from the twitter.statuses table into a Pandas Dataframe. I built this specific query for 3 reasons. First, I use distinct(status_id) to make sure there are not duplicate statuses. This is important because repeated entries can lead to slight biases and inconsistencies when testing, which is especially true for a smaller dataset. Second, I filter on language to only include english entries. Sentiment analysis typically works better when limited to one language, and English seemed like the obvious choice since it is my native language. Lastly, I use a subquery to filter out all the statuses containing links. I noticed while building this dataset that some statuses contain links to pictures or wesites, and some tweets contain JUST these links. Having links in the tweets will definitely throw off the accuracy of the sentiment analysis, so I left them out. This may lead to slight inconsistencies in the data (EX: What if one user has only tweeted images? Now they are not represented by the data at all), but I believe these inconsistencies are so minor they are negligible. I have printed the head (first 5 entries) for visualization of the data. 

In [118]:
res = cur.execute("""Select distinct(status_id), text 
                     from twitter.statuses s 
                     where s.lang = 'en' and status_id not in (
                         select status_id from twitter.statuses s2
                         where text like '%https://%') 
                     limit 1000"""
                 )
users = cur.fetchall()
df = pd.DataFrame.from_dict(users)
df.head()

,0,1
0,1000479232273743872,I don't know about you but I think Alden Ehren...
1,1000978713410859008,RT @mitmul: The PR for DLPack support in CuPy ...
2,1001167950232080385,RT @bonnienorman: @BretStephensNYT Oh ffs. Suc...
3,1001533153721413633,RT @egrefen: Super cool @PyTorch reimplementat...
4,1001549898247000064,RT @SpaceX: Now targeting June 1 launch of SES...


Below I have also displayed some basic data about the resulting dataframe.

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1000 non-null   object
 1   1       1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


We Can see the following information: there are 9 un-named columns of various data types in this dataframe. We are only concerned with the column representing the text of each tweet, which we can figure out is column index 6 from the .head() printout. In the next step we will process this dataframe.


## Step 3. 

To reformat the data, we will drop the columns we do not need, label the remaining column, and add a new labelled column for sentiment with all 0's.

In [120]:
# Drop all columns besides the 6th indexed
df = df.iloc[:, [1]]
# Add title to twitter text column
df = df.rename(columns={df.columns[0]: 'text'})
# Add a new column with zeros for sentiment target
df['target'] = 0

# Display info
print(f"Num rows: {df.shape[0]}")
df.head()


Num rows: 1000


,text,target
0,I don't know about you but I think Alden Ehren...,0
1,RT @mitmul: The PR for DLPack support in CuPy ...,0
2,RT @bonnienorman: @BretStephensNYT Oh ffs. Suc...,0
3,RT @egrefen: Super cool @PyTorch reimplementat...,0
4,RT @SpaceX: Now targeting June 1 launch of SES...,0


As seen in the above printout, we are now left with a Dataframe containing the information we want.


### Step 4. 

Now I will take the appropriate steps to export this dataframe to a CSV file named CustomTwitterSentimentiments.csv.

In [121]:
# Export DataFrame to a CSV file
df.to_csv('Data/CustomTwitterSentiments.csv', index=False)

### Steps 5 & 6. 

Now I will load back in the labelled CSV file in for some basic analysis. The data has been labelled with the following: -1 = negative, 0 = neutral, 1 = positive. I labelled the dataset this way to keep it identical in format to the Twitter_Data dataset. This is a dataset with 1.6 million tweets which I will use for training. 